# RDA5820 Functional Test 
- https://datasheet.lcsc.com/szlcsc/RDA-Microelectronics-RDA5820NS_C167244.pdf
- http://news.eeworld.com.cn/mcu/article_2017011333300.html

In [12]:
import os, sys

def gen_relative_path(target_path): 
    
    def del_same_parents(target_path, current_path):
        if len(target_path) * len(current_path) > 0:
            if target_path[0] == current_path[0]:
                target_path.pop(0)
                current_path.pop(0)
                del_same_parents(target_path, current_path)
            
    current_path = os.getcwd().replace('\\', '/').split('/')
    target_path = target_path.replace('\\', '/').split('/')
    del_same_parents(target_path, current_path)
    
    return ['..'] * len(current_path) + target_path


def append_source_relative_path(source_relative_path):
    sys.path.append(os.sep.join(source_relative_path))

In [13]:
# paths = [' ']
# paths = [gen_relative_path(p) for p in paths]
# print(paths)

In [14]:
paths =[['..', '..', '..', '..', '..', 'Signal Generators', 'bitbucket', 'github', 'codes'],
        ['..', '..', '..', '..', '..', 'FX2LP', 'bitbucket', 'github', 'codes'], 
        ['..', '..', '..', '..', '..', 'Utilities', 'bitbucket', 'github', 'codes'],
        ['..', '..', 'codes']]

for path in paths:
    append_source_relative_path(path)

In [15]:
%pylab inline
 
from array import array
from utilities.adapters import peripherals
from fm_transceivers import RDA5820
import fx2lp

Populating the interactive namespace from numpy and matplotlib


C:\Softwares\Python\lib\site-packages\IPython\core\magics\pylab.py:160: UserWarning: pylab import has clobbered these variables: ['array']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


## Debug mode?

In [16]:
cls = RDA5820

cls.DEBUG_MODE_SHOW_BUS_DATA = False         # whether to show bus data. 
cls.DEBUG_MODE_PRINT_REGISTER = False        # whether to print registers. 

## Generators

In [17]:
with_hardware_device = False

if with_hardware_device:
    _i2c = peripherals.I2C.get_Ftdi_i2c()
else:
    _i2c = None  # using None for testing without actual hardware device.

    
bus = peripherals.I2C(_i2c)


****** Virtual device. Data may not be real ! ******



In [18]:
bus = fx2lp.I2C(as_400KHz = True)

In [19]:
freq = 88.80e6
freq = 97.7e6

rda = RDA5820(bus, freq = freq)
initial_registers_values = rda.registers_values

## Compare configs

In [20]:
rda.read_all_registers()
read_back_registers_values = rda.registers_values

df = rda.map.compare_values_sets_pd(initial_registers_values, read_back_registers_values)
df[df.different == 1]

,register,address,default_value,element_name,idx_lowest_bit,n_bits,read_only,value,value_2,different
15,0x03,3,0,CHAN,6,10,False,0,319,1
21,0x04,4,17920,STCIEN,14,1,False,1,0,1
26,0x04,4,17920,SOFTMUTE_EN,9,1,False,1,0,1
39,0x05,5,34959,VOLUME,0,4,False,15,11,1
42,0x06,6,2048,SW_LR,11,1,False,1,0,1
52,0x07,7,19970,TH_SOFRBLEND,10,5,False,19,23,1
55,0x07,7,19970,SEEK_TH_OLD,2,6,False,0,49,1
63,0x0A,10,1024,ST,10,1,False,1,0,1
64,0x0A,10,1024,READCHAN,0,10,False,0,319,1
79,0x41,65,448,TXPA_VCOM,9,3,False,0,4,1


In [22]:
rda.power_down()

In [23]:
rda._soft_reset()
rda.power_up()

In [24]:
# rda._enable_i2s(True)
# rda.set_work_mode(mode = 'FM_Receiver')
rda.set_work_mode(mode = 'FM_Transmitter')
# rda.set_work_mode(mode = 'Audio_Amplifier')
# rda.set_work_mode(mode = 'CODEC')
# rda.set_work_mode(mode = 'ADC')

In [25]:
rda.set_frequency(88.8e6) 
rda.frequency // 1e4, rda.rssi

(8880.0, 85)

In [27]:
rda._set_band('US_Europe')

In [28]:
rda.stereo = True

In [29]:
rda._set_channel_spacing(100)

In [30]:
rda._enable_stc_interrupt(True)

In [31]:
rda._enable_audio_output(True)

In [ ]:
rda._enable_bass_boost(True)

In [32]:
rda.set_volume(15)

In [58]:
rda.mute(True)

In [59]:
rda.mute(False)

In [ ]:
rda.map.registers_by_address[5].print()

In [ ]:
rda.frequency

In [ ]:
seek_threshold = rda.seek_threshold
seek_threshold

In [ ]:
rda._set_seek_threshold(22)

In [ ]:
rda._seek_mode_wrap_around(True)
rda._seek_in_up_direction(True)
rda.seeking_is_in_up_direction
rda._set_seek_threshold(15)

In [ ]:
rda._seek()
rda.frequency // 1e4

In [ ]:
rda._seek_up()
rda.frequency // 1e4, rda.rssi

In [ ]:
rda._seek_down()
rda.frequency // 1e4, rda.rssi

In [ ]:
rda.read_all_registers()
baseline_registers_values = rda.registers_values

In [62]:
rda.set_frequency(88.9e6) 
rda.frequency // 1e4, rda.rssi

(8890.0, 85)

In [64]:
rda.set_frequency(88.8e6) 
rda.frequency // 1e4, rda.rssi

(8880.0, 85)

In [ ]:
rda._set_tx_pa_gain(15)

In [ ]:
rda.set_frequency(97.7e6) 
rda.frequency // 1e4, rda.rssi

In [ ]:
rda.read_all_registers()
read_back_registers_values = rda.registers_values

df = rda.map.compare_values_sets_pd(baseline_registers_values, read_back_registers_values)
df[df.different == 1]

In [ ]:
rda.stereo

In [ ]:
rda._is_fm_station

In [ ]:
rda._fm_ready

In [ ]:
rda.frequency

In [ ]:
rda.set_frequency(100.7e6) # 165
rda.frequency // 1e4, rda.rssi

In [ ]:
rda.set_frequency(103.5e6) # 165
rda.frequency // 1e4, rda.rssi

In [ ]:
rda.read_all_registers()
rda.print()

In [ ]:
rda._enable_afc(True)

In [68]:
rda._set_tx_pa_gain(63)

In [81]:
rda.stereo = False

In [80]:
rda.stereo = True

In [78]:
rda._set_fmtx_pga_gain( gain = 7)

In [77]:
rda._set_fmtx_adc_gain( gain = 7)

In [ ]:
# rda.map.reset()
# print(rda.registers_values)
# rda.write_all_registers()

## Noise Level Measurement

In [ ]:
plt.figure(figsize = (15, 5))
 
noise_levels = si.scan_noise_levels()
nl = np.array(noise_levels).T
plt.plot(nl[0] / 1e6, nl[1]);

In [ ]:
noise_levels_sorted = sorted(noise_levels, key = lambda x: x[1])
noise_levels_sorted[:30]

## RDS

In [ ]:
si.set_rds(program_id = 0x0520,
           station_name = "Wei Lin",
           radio_text = "My Radio Station !",
           program_type_code = 4,
           repeat_count = 3, message_count = 1, rds_mix_ratio = 50,  
           rds_fifo_size = 20,
           enable = True)

In [ ]:
# while True:
#     si.set_rds_buffer(message = time.strftime("%Y-%m-%d %H:%M:%S"), use_FIFO = True)
#     time.sleep(10)

In [ ]:
si.set_rds(enable = False)

## Member functions test

In [ ]:
# for f in dir(Si4713):
#     if not f.startswith('_'):
#         if f.islower():
#             if callable(getattr(Si4713, f)):
#                 print('si.{}()'.format(f))

In [ ]:
# si.apply_signal()

In [ ]:
si.close()

In [ ]:
si.start()

In [ ]:
si.enable(False)

In [ ]:
si.enable(True)

In [ ]:
si.enable_output(False)

In [ ]:
si.enable_output(True)

In [ ]:
# si.enable_output_channel(0)

In [ ]:
# si.load_registers()

In [ ]:
si.mute(True)

In [ ]:
si.mute(False)

In [ ]:
si.mute_line_input(True)

In [ ]:
si.stereo = False
si.stereo

In [ ]:
si.stereo = True
si.stereo

In [ ]:
si.mute_line_input(False)

In [ ]:
si._set_line_input_level(attenuation_level = 3)
si.line_level

In [ ]:
si._set_line_input_level(attenuation_level = 0)
si.line_level

In [ ]:
si.pause()

In [ ]:
si.resume()

In [ ]:
si.power_down()

In [ ]:
si.power_up()

In [ ]:
si.init()

In [ ]:
si.power_down()

In [ ]:
si.reset()

In [ ]:
# si.select_freq_source(0)

In [ ]:
# si.select_phase_source(0)

In [ ]:
si.set_frequency(88.9e6)

In [ ]:
si.set_frequency(88.8e6)

In [ ]:
si.set_gpio_direction()

In [ ]:
si.set_gpio_level()

In [ ]:
# si.set_phase(0)

In [ ]:
si.set_power(88)

In [ ]:
si.set_power(115)

In [ ]:
si.stop()

In [ ]:
si.start()

In [ ]:
si.update()

In [ ]:
si.read_all_registers()

In [ ]:
si.write_all_registers()

In [ ]:
si.print()

In [ ]:
DEFAULT_REGISTERS_VALUES = ((1, 199), (257, 0), (259, 0), (513, 32768), (514, 1), (8448, 3), (8449, 6625),
                            (8450, 675), (8451, 0), (8452, 190), (8453, 0), (8454, 0), (8455, 19000), (8704, 3),
                            (8705, 65496), (8706, 2), (8707, 4), (8708, 15), (8709, 13), (8960, 7), (8961, 206),
                            (8962, 10000), (8963, 236), (8964, 5000), (11264, 0), (11265, 0), (11266, 0),
                            (11267, 0), (11268, 0), (11269, 0), (11270, 0), (11271, 0))

In [ ]:
si.init()
initial_registers_values = si.registers_values

In [ ]:
si.read_all_registers()
read_back_registers_values = si.registers_values

In [ ]:
df = si.map.compare_values_sets_pd(initial_registers_values, DEFAULT_REGISTERS_VALUES)
df[df.different == 1]

In [ ]:
df = si.map.compare_values_sets_pd(initial_registers_values, read_back_registers_values)
# df
df[df.different == 1]